In [5]:
import numpy as np
np.random.seed(123)

> 2번 base의 전처리가 다음 문제에 영향을 끼치지 않는다. 2번 문제 풀 때 base랑 3번 문제 풀 때 base는 같다

▶ 삼성전자의 고객 마케팅에 활용하기 위하여 삼성 카드 측에서 비식별화된 고객 데이터를 협조 받아 통합 분석을 하기 위한 선제 분석을 하고자 한다.  
※ 분석 수행 전 ‘신용카드 한도(CREDIT_LIMIT)’와 ‘기한 내 최소 지불 금액(MINIMUM_PAYMENTS)’의 결측 값(Null)을 각 컬럼의 평균값으로 대체하시오.

전처리 수행 결과를 base 객체로 지칭하고 다음 문항에서 해당 객체를 기반으로 문제를 풀이하시오.

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.tree import DecisionTreeRegressor

In [7]:
import pandas as pd
df_origin = pd.read_csv("../Datasets/card_cust.csv")

In [9]:
df_origin.head(3)

CUST_ID      BALANCE  BALANCE_FREQUENCY  PURCHASES  ONEOFF_PURCHASES  \
0    10001    40.900749           0.818182      95.40              0.00   
1    10002  3202.467416           0.909091       0.00              0.00   
2    10003  2495.148862           1.000000     773.17            773.17   

   INSTALLMENTS_PURCHASES  CASH_ADVANCE  PURCHASES_FREQUENCY  \
0                    95.4      0.000000             0.166667   
1                     0.0   6442.945483             0.000000   
2                     0.0      0.000000             1.000000   

   ONEOFF_PURCHASES_FREQUENCY  PURCHASES_INSTALLMENTS_FREQUENCY  \
0                         0.0                          0.083333   
1                         0.0                          0.000000   
2                         1.0                          0.000000   

   CASH_ADVANCE_FREQUENCY  CASH_ADVANCE_TRX  PURCHASES_TRX  CREDIT_LIMIT  \
0                    0.00               0.0            2.0        1000.0   
1                    0.25               4.0            0.0        7000.0   
2                    0.00               0.0           12.0        7500.0   

      PAYMENTS  MINIMUM_PAYMENTS  PRC_FULL_PAYMENT  TENURE  
0   201.802084        139.509787          0.000000    12.0  
1  4103.032597       1072.340217          0.222222    12.0  
2   622.066742        627.284787          0.000000    12.0

In [10]:
df_origin["CREDIT_LIMIT"] = df_origin["CREDIT_LIMIT"].fillna(df_origin["CREDIT_LIMIT"].mean())
df_origin["MINIMUM_PAYMENTS"] = df_origin["MINIMUM_PAYMENTS"].fillna(df_origin["MINIMUM_PAYMENTS"].mean())

In [11]:
df = df_origin.copy()

In [12]:
df.isna().sum()

CUST_ID                             0
BALANCE                             0
BALANCE_FREQUENCY                   0
PURCHASES                           0
ONEOFF_PURCHASES                    0
INSTALLMENTS_PURCHASES              0
CASH_ADVANCE                        0
PURCHASES_FREQUENCY                 0
ONEOFF_PURCHASES_FREQUENCY          0
PURCHASES_INSTALLMENTS_FREQUENCY    0
CASH_ADVANCE_FREQUENCY              0
CASH_ADVANCE_TRX                    0
PURCHASES_TRX                       0
CREDIT_LIMIT                        0
PAYMENTS                            0
MINIMUM_PAYMENTS                    0
PRC_FULL_PAYMENT                    0
TENURE                              0
dtype: int64

#### Q1. (base를 사용하여)연간 평균 잔고액과 신용카드 서비스 이용기간 간의 관계를 파악하여, 추후 고객의 신용카드 한도 조정에 근거 자료로 활용하고자 한다. 연간 평균 잔고액(BALANCE)이 많을수록, 그리고 신용카드 서비스 이용기간(TENURE)이 길수록 신용카드 한도(CREDIT_LIMIT) 역시 높을 것으로 예상해볼 수 있다. 신용 카드 서비스 이용기간(TENURE) 별로 연간 평균 잔고액(BALANCE)과 신용카드 한도(CREDIT_LIMIT) 간 피어슨(Pearson) 상관 분석을 실시하고, 이 중 가장 큰 상관계수를 구하시오.
※ 정답은 반올림하여 소수점 둘째 자리까지 출력하시오.  
(정답 예시: 0.12)

In [13]:
df["TENURE"].unique()

array([12.,  8., 11.,  9., 10.,  7.,  6.])

In [14]:
df[["BALANCE", "CREDIT_LIMIT"]].corr().iloc[0, 1]

0.46764597335688984

In [15]:
ls_corr = []
for n in df["TENURE"].unique():
    _df = df.loc[df["TENURE"] == n, ["BALANCE", "CREDIT_LIMIT"]]
    stat_corr = _df.corr().iloc[0, 1]
    ls_corr = ls_corr + [stat_corr]

In [16]:
round(max(ls_corr), 2)

0.95

Q. groupby로도 풀어봐요

In [17]:
df

CUST_ID      BALANCE  BALANCE_FREQUENCY  PURCHASES  ONEOFF_PURCHASES  \
0      10001    40.900749           0.818182      95.40              0.00   
1      10002  3202.467416           0.909091       0.00              0.00   
2      10003  2495.148862           1.000000     773.17            773.17   
3      10004  1666.670542           0.636364    1499.00           1499.00   
4      10005   817.714335           1.000000      16.00             16.00   
..       ...          ...                ...        ...               ...   
995    11029  1250.394614           0.909091     443.99            443.99   
996    11030     9.503968           1.000000      96.62              0.00   
997    11031  2285.068731           1.000000       0.00              0.00   
998    11032  2928.756699           1.000000     160.92              0.00   
999    11033  1460.178578           1.000000     937.96            100.00   

     INSTALLMENTS_PURCHASES  CASH_ADVANCE  PURCHASES_FREQUENCY  \
0                     95.40      0.000000             0.166667   
1                      0.00   6442.945483             0.000000   
2                      0.00      0.000000             1.000000   
3                      0.00    205.788017             0.083333   
4                      0.00      0.000000             0.083333   
..                      ...           ...                  ...   
995                    0.00      0.000000             0.272727   
996                   96.62      0.000000             1.000000   
997                    0.00   1173.310874             0.000000   
998                  160.92    319.931964             1.000000   
999                  837.96      0.000000             1.000000   

     ONEOFF_PURCHASES_FREQUENCY  PURCHASES_INSTALLMENTS_FREQUENCY  \
0                      0.000000                          0.083333   
1                      0.000000                          0.000000   
2                      1.000000                          0.000000   
3                      0.083333                          0.000000   
4                      0.083333                          0.000000   
..                          ...                               ...   
995                    0.272727                          0.000000   
996                    0.000000                          1.000000   
997                    0.000000                          0.000000   
998                    0.000000                          1.000000   
999                    0.083333                          1.000000   

     CASH_ADVANCE_FREQUENCY  CASH_ADVANCE_TRX  PURCHASES_TRX  CREDIT_LIMIT  \
0                  0.000000               0.0            2.0        1000.0   
1                  0.250000               4.0            0.0        7000.0   
2                  0.000000               0.0           12.0        7500.0   
3                  0.083333               1.0            1.0        7500.0   
4                  0.000000               0.0            1.0        1200.0   
..                      ...               ...            ...           ...   
995                0.000000               0.0            3.0        3500.0   
996                0.000000               0.0           19.0        4500.0   
997                0.166667               5.0            0.0        2500.0   
998                0.333333               5.0           12.0        3000.0   
999                0.000000               0.0           40.0        1500.0   

        PAYMENTS  MINIMUM_PAYMENTS  PRC_FULL_PAYMENT  TENURE  
0     201.802084        139.509787          0.000000    12.0  
1    4103.032597       1072.340217          0.222222    12.0  
2     622.066742        627.284787          0.000000    12.0  
3       0.000000       1297.116322          0.000000    12.0  
4     678.334763        244.791237          0.000000    12.0  
..           ...               ...               ...     ...  
995   273.823646        259.715939          0.000000    11.0  
996  1086.932525 

In [18]:
df_corr = df.groupby("TENURE")[["BALANCE", "CREDIT_LIMIT"]].corr()
df_corr

BALANCE  CREDIT_LIMIT
TENURE                                     
6.0    BALANCE       1.000000      0.868056
       CREDIT_LIMIT  0.868056      1.000000
7.0    BALANCE       1.000000      0.948405
       CREDIT_LIMIT  0.948405      1.000000
8.0    BALANCE       1.000000      0.820696
       CREDIT_LIMIT  0.820696      1.000000
9.0    BALANCE       1.000000      0.085474
       CREDIT_LIMIT  0.085474      1.000000
10.0   BALANCE       1.000000      0.291482
       CREDIT_LIMIT  0.291482      1.000000
11.0   BALANCE       1.000000      0.380360
       CREDIT_LIMIT  0.380360      1.000000
12.0   BALANCE       1.000000      0.460833
       CREDIT_LIMIT  0.460833      1.000000

In [19]:
df_corr.unstack()['BALANCE']['CREDIT_LIMIT']

TENURE
6.0     0.868056
7.0     0.948405
8.0     0.820696
9.0     0.085474
10.0    0.291482
11.0    0.380360
12.0    0.460833
Name: CREDIT_LIMIT, dtype: float64

In [20]:
df_corr.unstack()['BALANCE']['CREDIT_LIMIT'].round(2).max()

0.95

#### Q2. (base를 사용하여)전략을 수립하기 위해 고객 세분화를 수행하고자 한다. 일시불 구매 금액이 높은 고객군을 도출하기 위해 다음 단계에 따라 분석을 수행하고 질문에 답하시오.

* 단계 1: ‘고객 ID’를 제외한 모든 변수(17개)에 대해 Z-score 표준화(Standardization) 한다. 
* 단계 2: 표준화된 변수들에 대해 K-means 군집 분석을 수행한다. 이 때, 군집 수는 2~5개 중 K-means Silhouette 를 통해 구한 최적의 K로 설정한다.
* 단계 3: 단계 2에서 도출한 각 군집 별로 ‘일시불 구매 총액’의 평균을 계산한다.

군집 별 일시불 구매 총액(ONEOFF_PURCHASES)의 평균 중 가장 큰 값은 얼마인가?

※ seed는 1234로 설정하시오.  
※ 정답은 반올림하여 소수점 둘째 자리까지 출력하시오.  
(정답 예시: 0.12)

※ seed는 1234로 설정하시오.  

In [21]:
seed = 1234

In [22]:
df = df_origin.copy()

* 단계 1: ‘고객 ID’를 제외한 모든 변수(17개)에 대해 Z-score 표준화(Standardization) 한다. 

In [23]:
df = df.drop("CUST_ID", axis = 1)

In [24]:
df2 = StandardScaler().fit_transform(df)
df2 = pd.DataFrame(df2, columns = df.columns)

In [25]:
df2.head(1)

BALANCE  BALANCE_FREQUENCY  PURCHASES  ONEOFF_PURCHASES  \
0 -0.848768          -0.419879  -0.441936         -0.374048   

   INSTALLMENTS_PURCHASES  CASH_ADVANCE  PURCHASES_FREQUENCY  \
0               -0.395301     -0.482354            -0.872701   

   ONEOFF_PURCHASES_FREQUENCY  PURCHASES_INSTALLMENTS_FREQUENCY  \
0                   -0.804321                          -0.71962   

   CASH_ADVANCE_FREQUENCY  CASH_ADVANCE_TRX  PURCHASES_TRX  CREDIT_LIMIT  \
0               -0.684701         -0.457918      -0.564116     -1.161669   

   PAYMENTS  MINIMUM_PAYMENTS  PRC_FULL_PAYMENT    TENURE  
0 -0.557396         -0.443725         -0.465544  0.282429

* 단계 2: 표준화된 변수들에 대해 K-means 군집 분석을 수행한다. 이 때, 군집 수는 2~5개 중 K-means Silhouette 를 통해 구한 최적의 K로 설정한다.

In [26]:
ls_score = []
for k in range(2, 6):
    model_km = KMeans(n_clusters = k, random_state = seed)
    model_km.fit(df2)
    score_sub = silhouette_score(df2, model_km.labels_)
    ls_score = ls_score + [score_sub]

C:\Users\beluga\miniconda3\envs\s0921\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\beluga\miniconda3\envs\s0921\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(
C:\Users\beluga\miniconda3\envs\s0921\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\beluga\miniconda3\envs\s0921\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than availabl

In [27]:
ser_score = pd.Series(ls_score, index = range(2, 6))
best_k = ser_score.idxmax()
best_k

2

In [28]:
model = KMeans(n_clusters = best_k, random_state = seed)
model.fit(df2)

C:\Users\beluga\miniconda3\envs\s0921\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\beluga\miniconda3\envs\s0921\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=4.
  warnings.warn(


KMeans(n_clusters=2, random_state=1234)

* 단계 3: 단계 2에서 도출한 각 군집 별로 ‘일시불 구매 총액’의 평균을 계산한다.

군집 별 일시불 구매 총액(ONEOFF_PURCHASES)의 평균 중 가장 큰 값은 얼마인가?


※ 정답은 반올림하여 소수점 둘째 자리까지 출력하시오.  
(정답 예시: 0.12)

In [29]:
df["cluster"] = model.labels_
df.groupby("cluster")["ONEOFF_PURCHASES"].mean().round(2).max()

3946.19

#### Q3. (base를 사용하여) 일시불 구매 총액(ONEOFF_PURCHASES) 예측 모델을 Target Marketing에 활용하고자 한다. 다음 단계에 따라 분석을 수행하고 질문에 답하시오.

* 단계 1: ‘고객 ID(CUST_ID)’가 4의 배수가 아닌 데이터를 Train Set으로, 4의 배수인 데이터를 Test Set으로 분할한다. 
* 단계 2: Train Set으로 아래 조건에 따라 의사결정나무 회귀모델을 학습한다. 
  - 독립 변수(총 16개):‘고객 ID’,‘일시불 구매 총액’을 제외한 모든 변수
  - 종속 변수: ‘일시불 구매 총액’
* 단계 3: 생성된 모델을 Test Set에 적용하여 ‘일시불 구매 총액’을 예측한다.

단계 3에서 얻은 예측 결과를 평가하기 위해, 아래 정의된 Measure **B**를 계산한 값은?
$$ B = \biggl(\frac{1}{n} \sum_{i=1}^{n}{(y_i - \hat{y_i})^2}\biggr)^{\frac{1}{2}}, \quad \hat{y_i}: 예측값, y_i: 실제값 $$

※ seed는 1234로 지정하시오.  
※ 정답은 반올림하여 소수점 첫째 자리까지 출력하시오.  
(정답 예시: 0.1)

In [30]:
seed = 1234
df = df_origin.copy()

* 단계 1: ‘고객 ID(CUST_ID)’가 4의 배수가 아닌 데이터를 Train Set으로, 4의 배수인 데이터를 Test Set으로 분할한다. 

In [31]:
df_train = df.loc[df["CUST_ID"] % 4 != 0]
df_test  = df.loc[df["CUST_ID"] % 4 == 0]
len(df_train), len(df_test)

(752, 248)

In [32]:
df_train=df_train.drop(columns='CUST_ID')
df_test=df_test.drop(columns='CUST_ID')

* 단계 2: Train Set으로 아래 조건에 따라 의사결정나무 회귀모델을 학습한다. 
  - 독립 변수(총 16개):‘고객 ID’,‘일시불 구매 총액’을 제외한 모든 변수
  - 종속 변수: ‘일시불 구매 총액’ 

In [33]:
model_dtr = DecisionTreeRegressor(random_state = seed)
model_dtr.fit(X = df_train.drop(columns="ONEOFF_PURCHASES"),
          y = df_train["ONEOFF_PURCHASES"])

DecisionTreeRegressor(random_state=1234)

* 단계 3: 생성된 모델을 Test Set에 적용하여 ‘일시불 구매 총액’을 예측한다.

단계 3에서 얻은 예측 결과를 평가하기 위해, 아래 정의된 Measure **B**를 계산한 값은?
※ 정답은 반올림하여 소수점 첫째 자리까지 출력하시오.  
(정답 예시: 0.1)

In [34]:
y_pred = model_dtr.predict(df_test.drop(columns="ONEOFF_PURCHASES"))

In [35]:
y_true = df_test["ONEOFF_PURCHASES"]

In [36]:
round(((y_true - y_pred) ** 2).mean() ** 0.5, 1)

1039.2

In [37]:
from sklearn.metrics import mean_squared_error
round(mean_squared_error(y_true = y_true, y_pred = y_pred) ** 0.5, 1)

1039.2